# Импорт библиотек

import pickle
import random
import time
import os
import shutil
import copy

import numpy as np
import pandas as pd

from skimage import io
from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path
#import torchvision.transforms as transforms

from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from __future__ import print_function, division
from matplotlib import colors, pyplot as plt
%matplotlib inline
plt.ion()

import torch
import torch.nn as nn
import torchvision.models as models
import torchsummary
from torch.utils.data import datasets, DataLoader
import torchvision
from torchvision import datasets, transforms, models
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
from torch.autograd import Variable
import torch.onnx

In [1]:
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
import numpy as np
import torchvision
import random

from torchvision import datasets, transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
import torch.nn.functional as F
import torch.optim as optim
import torch.onnx

In [2]:
DATA_MODES = ['train', 'val', 'test']
RESCALE_SIZE = 224
EPOCHS = 10
BATCH_SIZE = 16
LEARNING_RATE = 0.001

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Модели

## Модель `create_nn_v2`

In [3]:
data_dir = './data/classed_nrj_train/'

In [4]:
#Набор трансформаций для обучающей выборки
train_transforms = transforms.Compose([transforms.CenterCrop(224), 
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomVerticalFlip(p=0.5),
                                       transforms.RandomRotation(degrees=(45, 90)),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406],
                                            [0.229, 0.224, 0.225])
                                       ])

In [5]:
#Набор трансформаций для валидационной выборки
valid_transforms = transforms.Compose([transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize(
                                           [0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                                       ])

In [6]:
#Набор трансформаций для тестовой выборки
test_transforms = transforms.Compose([transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          [0.485, 0.456, 0.406],
                                           [0.229, 0.224, 0.225])
                                      ])

In [7]:
#Объявляю функцию загрузчика с аргументами:
#Директория, Размер батча, Набор трансформаций для тренировочной выборки,
#Набор трансформаций для валидационной выборки, Размер валидационной выборки
def load_split_train_valid(datadir,
                           batch_size,
                           train_transforms,
                           valid_transforms,
                           valid_size): 
    
    #Загрузчик для тренировочной выборки
    train_data = datasets.ImageFolder(datadir,       
                    transform=train_transforms)
    #Загрузчик для тренировочной выборки
    test_data = datasets.ImageFolder(datadir,
                    transform=valid_transforms)
    #Код для разделения на трейн и тест в указанном соотношении
    num_train = len(train_data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))
    np.random.shuffle(indices)
    from torch.utils.data.sampler import SubsetRandomSampler
    train_idx, test_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    trainloader = torch.utils.data.DataLoader(train_data,
                   sampler=train_sampler, batch_size=batch_size)
    testloader = torch.utils.data.DataLoader(test_data,
                   sampler=test_sampler, batch_size=batch_size)
    return trainloader, testloader

In [8]:
#Получаю тренировочный и валидационный генераторы
train_loader, val_loader = load_split_train_valid(datadir=data_dir,
                           batch_size=BATCH_SIZE,
                           train_transforms=train_transforms,
                           valid_transforms=valid_transforms,
                           valid_size = .2)
#Проверяю результаты работы генераторов
print(train_loader.dataset.classes)
print(val_loader.dataset.classes)
print(len(train_loader))
print(len(val_loader))

['1', '10', '20', '3', '30', '6']
['1', '10', '20', '3', '30', '6']
664
166


In [9]:
class ConvNet(nn.Module): 
        def __init__(self):
            super(ConvNet, self).__init__()
            self.c1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
            self.c2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
            self.c3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
            self.fc1 = nn.Linear(in_features=6272, out_features=1024)
            self.fc2 = nn.Linear(in_features=1024, out_features=512)
            self.fc3 = nn.Linear(in_features=512, out_features=256)
            self.fc4 = nn.Linear(in_features=256, out_features=6)
        
        def forward(self, x):
            x = F.relu(F.max_pool2d(self.c1(x), 3))
            x = F.relu(F.max_pool2d(self.c2(x), 3))
            x = F.relu(F.max_pool2d(self.c3(x), 3))
            x = x.view(x.size(0), -1)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            x = self.fc4(x)

            return F.log_softmax(x, dim=1)

In [10]:
net = ConvNet()
print(net)
net = net.cuda()

ConvNet(
  (c1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
  (c2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (c3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=6272, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=6, bias=True)
)


In [11]:
#Задаю функцию потерь
criterion = nn.CrossEntropyLoss()
#Задаю оптимизатор
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)

total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in tqdm(range(EPOCHS)):
    for i, (images, labels) in enumerate(train_loader):
    #Прямой запуск
        images=images.to(device)
        labels=labels.to(device)
            
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        #Обратное распространение и оптимизатор
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        #Отслеживание точности
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
            .format(epoch + 1, EPOCHS, i + 1, total_step, loss.item(),
                                  (correct / total) * 100))


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Epoch [1/10], Step [100/664], Loss: 1.3224, Accuracy: 62.50%
Epoch [1/10], Step [200/664], Loss: 0.2548, Accuracy: 93.75%
Epoch [1/10], Step [300/664], Loss: 0.3418, Accuracy: 81.25%
Epoch [1/10], Step [400/664], Loss: 0.2114, Accuracy: 93.75%
Epoch [1/10], Step [500/664], Loss: 0.0807, Accuracy: 100.00%
Epoch [1/10], Step [600/664], Loss: 0.0549, Accuracy: 100.00%


 10%|████████▏                                                                         | 1/10 [04:21<39:17, 261.97s/it]

Epoch [2/10], Step [100/664], Loss: 0.0112, Accuracy: 100.00%
Epoch [2/10], Step [200/664], Loss: 0.6988, Accuracy: 81.25%
Epoch [2/10], Step [300/664], Loss: 0.0315, Accuracy: 100.00%
Epoch [2/10], Step [400/664], Loss: 0.2736, Accuracy: 93.75%
Epoch [2/10], Step [500/664], Loss: 0.0036, Accuracy: 100.00%
Epoch [2/10], Step [600/664], Loss: 0.3615, Accuracy: 87.50%


 20%|████████████████▍                                                                 | 2/10 [05:20<18:59, 142.40s/it]

...

 80%|██████████████████████████████████████████████████████████████████▍                | 8/10 [11:04<02:05, 62.67s/it]

Epoch [9/10], Step [100/664], Loss: 0.0003, Accuracy: 100.00%
Epoch [9/10], Step [200/664], Loss: 0.0010, Accuracy: 100.00%
Epoch [9/10], Step [300/664], Loss: 0.0006, Accuracy: 100.00%
Epoch [9/10], Step [400/664], Loss: 0.5328, Accuracy: 93.75%
Epoch [9/10], Step [500/664], Loss: 0.0058, Accuracy: 100.00%
Epoch [9/10], Step [600/664], Loss: 0.0009, Accuracy: 100.00%


 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [12:02<01:01, 61.33s/it]

Epoch [10/10], Step [100/664], Loss: 0.0001, Accuracy: 100.00%
Epoch [10/10], Step [200/664], Loss: 0.0002, Accuracy: 100.00%
Epoch [10/10], Step [300/664], Loss: 0.0082, Accuracy: 100.00%
Epoch [10/10], Step [400/664], Loss: 0.0011, Accuracy: 100.00%
Epoch [10/10], Step [500/664], Loss: 0.0080, Accuracy: 100.00%
Epoch [10/10], Step [600/664], Loss: 0.0018, Accuracy: 100.00%


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [12:59<00:00, 77.98s/it]


In [12]:
#Проверка на валидационной выборке
net.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in tqdm(val_loader):
        images=images.to(device)
        labels=labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model on the test images: {} %'
              .format((correct / total) * 100))

100%|████████████████████████████████████████████████████████████████████████████████| 166/166 [00:55<00:00,  2.98it/s]

Test Accuracy of the model on the test images: 99.73644578313254 %
